# 🎮 Phase 4: Atari 환경 실습

> **목표**: Atari 게임 환경에서 Decision Transformer가 어떻게 작동하는지 이해하고,  
> 이미지 기반 강화학습의 특수성을 학습합니다.

---

## 🗺️ 이 Phase에서 배울 내용

```
┌────────────────────────────────────────────────────────────────────────────┐
│                      Phase 4: Atari 환경 실습 개요                          │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│   ┌──────────────┐    ┌──────────────┐    ┌──────────────┐                │
│   │   DQN        │    │   CNN        │    │   Atari      │                │
│   │  Replay      │ ──▶│  Encoder     │ ──▶│   GPT        │                │
│   │  데이터셋    │    │  (이미지→벡터)│    │  모델        │                │
│   └──────────────┘    └──────────────┘    └──────────────┘                │
│         │                   │                    │                        │
│         ▼                   ▼                    ▼                        │
│   • 84×84 픽셀 이미지      • Conv 레이어       • Reward-Conditioned      │
│   • 4 프레임 스택          • 특징 추출         • Naive (BC)               │
│   • 50개 버퍼              • 3136→128 압축    • Cross-Entropy Loss       │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
```

## 📋 목차

| 섹션 | 주제 | 학습 내용 |
|:---:|:---|:---|
| **1** | [환경 설정](#1-환경-설정) | 경로 설정 및 데이터 확인 |
| **2** | [데이터셋 생성 분석](#2-데이터셋-생성-분석) | DQN Replay Buffer와 RTG 계산 |
| **3** | [CNN Encoder 이해](#3-cnn-encoder-이해) | 이미지를 벡터로 변환하는 과정 |
| **4** | [모델 구조 분석](#4-모델-구조-분석) | Atari용 GPT 모델 구조 |
| **5** | [Reward-Conditioned vs Naive](#5-reward-conditioned-vs-naive-비교) | 두 모드의 차이점 |

---

## 🔗 Phase 연결

```
Phase 1 (배경지식)     Phase 2 (핵심개념)     Phase 3 (Gym)      Phase 4 (Atari)
      │                     │                    │                   │
      ▼                     ▼                    ▼                   ▼
  RL/Transformer       RTG/시퀀스           연속 상태 ──▶     이미지 상태
    기초 개념          구성 원리            (11차원 벡터)    (84×84×4 픽셀)
```

## 🎯 Gym vs Atari 비교

| 항목 | Gym (Phase 3) | Atari (Phase 4) |
|:---|:---|:---|
| **State** | 11차원 벡터 | 84×84×4 이미지 |
| **Action** | 연속값 (3차원) | 이산값 (4~18개) |
| **손실 함수** | MSE | Cross-Entropy |
| **Encoder** | Linear | CNN |
| **데이터셋** | D4RL | DQN Replay |

---

## 📦 사전 요구사항

```python
# 필요한 패키지
- Python 3.7+
- PyTorch 1.8+
- NumPy
- Matplotlib

# 선택적 (실제 데이터 사용 시)
- gsutil (Google Cloud Storage 접근용)
- Atari ROMs
```

In [ ]:
# ============================================================
# 📦 필수 패키지 임포트 및 환경 설정
# ============================================================
# Atari Decision Transformer 분석에 필요한 패키지들입니다.
#
# 주요 패키지:
#   - torch: 딥러닝 프레임워크 (CNN, Transformer)
#   - numpy: 이미지 데이터 처리
#   - matplotlib: 시각화
# ============================================================

import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 재현성을 위한 시드 설정
np.random.seed(42)
torch.manual_seed(42)

print("🔧 환경 정보")
print("="*50)
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")
print(f"   NumPy: {np.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   CUDA device: {torch.cuda.get_device_name(0)}")

---
# 1. 환경 설정

## 🎯 목표
> Atari Decision Transformer 코드에 접근하기 위한 경로를 설정하고, 데이터 구조를 확인합니다.

## 📂 Atari 프로젝트 구조

```
decision-transformer/
├── atari/                          ← 이번 Phase에서 분석할 코드
│   ├── mingpt/
│   │   ├── model_atari.py          ← 핵심: GPT 모델 + CNN Encoder
│   │   ├── trainer_atari.py        ← 학습 루프
│   │   └── utils.py
│   ├── run_dt_atari.py             ← 메인 실행 스크립트
│   ├── create_dataset.py           ← DQN replay → 학습 데이터 변환
│   ├── fixed_replay_buffer.py      ← Replay buffer 로딩
│   └── dqn_replay/                 ← 데이터셋 (다운로드 필요)
│       ├── Breakout/
│       ├── Pong/
│       └── ...
└── doc/notebooks/                  ← 현재 위치
```

## 🎮 지원되는 Atari 게임

| 게임 | Action 수 | 특징 |
|:---|:---:|:---|
| Breakout | 4 | 벽돌깨기, 간단한 액션 |
| Pong | 6 | 탁구, 2인 대전 |
| Qbert | 6 | 플랫폼 점프 |
| Seaquest | 18 | 복잡한 액션 공간 |

In [ ]:
# ============================================================
# 📂 프로젝트 경로 설정
# ============================================================
# Python이 atari/ 폴더의 모듈을 찾을 수 있도록 경로를 추가합니다.
# ============================================================

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
ATARI_PATH = os.path.join(PROJECT_ROOT, 'atari')

if ATARI_PATH not in sys.path:
    sys.path.insert(0, ATARI_PATH)

print("📂 경로 설정 결과")
print("="*50)
print(f"   Project root: {PROJECT_ROOT}")
print(f"   Atari path:   {ATARI_PATH}")
print()

# 경로 검증
if os.path.exists(ATARI_PATH):
    print("✅ Atari 경로 존재 확인!")
    mingpt_path = os.path.join(ATARI_PATH, 'mingpt')
    if os.path.exists(mingpt_path):
        print("✅ mingpt 모듈 존재 확인!")
        # 주요 파일 확인
        key_files = ['model_atari.py', 'trainer_atari.py']
        for f in key_files:
            fpath = os.path.join(mingpt_path, f)
            status = "✅" if os.path.exists(fpath) else "❌"
            print(f"   {status} {f}")
else:
    print("❌ Atari 경로가 존재하지 않습니다!")

In [ ]:
# ============================================================
# 🎮 데이터셋 디렉토리 확인
# ============================================================
# DQN replay 데이터셋이 저장되는 위치를 확인합니다.
#
# 데이터셋 다운로드 방법:
#   gsutil -m cp -R gs://atari-replay-datasets/dqn/Breakout dqn_replay/
#
# 각 게임 폴더에는 50개의 checkpoint가 있습니다 (1/, 2/, ..., 50/)
# ============================================================

data_dir = os.path.join(ATARI_PATH, 'dqn_replay')

print("📂 데이터셋 확인")
print("="*50)

if os.path.exists(data_dir):
    games = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    if games:
        print(f"\n✅ 사용 가능한 게임 ({len(games)}개):\n")
        for game in sorted(games):
            game_path = os.path.join(data_dir, game)
            checkpoints = [d for d in os.listdir(game_path) if d.isdigit()]
            print(f"   🎮 {game}")
            print(f"      └─ {len(checkpoints)}개 checkpoint")
    else:
        print("\n⚠️ 게임 데이터가 없습니다.")
else:
    print(f"❌ 데이터 디렉토리가 없습니다: {data_dir}")

print("\n" + "-"*50)
print("📥 데이터셋 다운로드 방법:")
print("   mkdir -p dqn_replay")
print("   gsutil -m cp -R gs://atari-replay-datasets/dqn/Breakout dqn_replay/")
print("\n💡 gsutil이 없다면 Google Cloud SDK를 설치하세요.")

---
# 2. 데이터셋 생성 분석

## 🎯 목표
> DQN Replay Buffer의 구조와 Atari 스타일 RTG 계산 방식을 이해합니다.

## 📚 DQN Replay Buffer란?

**DQN (Deep Q-Network)**이 학습하면서 저장한 경험 데이터입니다.

```
┌────────────────────────────────────────────────────────────────────────────┐
│                       DQN Replay Buffer 구조                               │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  DQN 학습 과정:                                                            │
│  ┌──────────────────────────────────────────────────────────────────────┐ │
│  │                                                                      │ │
│  │  checkpoint 1/  ──▶ 100,000 transitions (초보 정책)                 │ │
│  │  checkpoint 2/  ──▶ 100,000 transitions                             │ │
│  │  checkpoint 3/  ──▶ 100,000 transitions                             │ │
│  │       ...                                                            │ │
│  │  checkpoint 50/ ──▶ 100,000 transitions (전문가 정책)               │ │
│  │                                                                      │ │
│  └──────────────────────────────────────────────────────────────────────┘ │
│                                                                            │
│  각 transition:                                                            │
│    (observation, action, reward, terminal)                                │
│         │           │       │        │                                    │
│         │           │       │        └─ 에피소드 종료 여부               │
│         │           │       └─ 즉시 보상                                 │
│         │           └─ 수행한 행동 (0~17)                                │
│         └─ 게임 화면 (4 stacked frames, 84×84)                          │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
```

## 💡 왜 50개 버퍼를 사용할까?

| 버퍼 번호 | 정책 품질 | 특징 |
|:---:|:---|:---|
| 1~10 | 낮음 (초보) | 랜덤에 가까운 행동 |
| 11~30 | 중간 | 학습 중인 정책 |
| 31~50 | 높음 (전문가) | 최적화된 정책 |

> 💡 **다양한 품질의 데이터**를 사용하면 DT가 더 잘 학습합니다!  
> RTG 조건에 따라 초보~전문가 수준의 행동을 모두 학습할 수 있습니다.

## 2.1 DQN Replay Buffer 구조 상세

### 📦 Observation의 구조

Atari 게임의 observation은 **4개 프레임을 쌓은(stack)** 이미지입니다.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    4 Frame Stack의 의미                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   왜 4개 프레임을 쌓을까?                                            │
│   ───────────────────────                                           │
│   단일 프레임만으로는 물체의 "움직임 방향"을 알 수 없습니다.          │
│                                                                     │
│   예: Pong 게임에서 공의 위치만으로는 공이 어디로 가는지 모름        │
│                                                                     │
│   Frame 1     Frame 2     Frame 3     Frame 4                       │
│   ┌─────┐     ┌─────┐     ┌─────┐     ┌─────┐                       │
│   │  ○  │     │   ○ │     │    ○│     │     │○                      │
│   │     │ ──▶ │     │ ──▶ │     │ ──▶ │     │                       │
│   │     │     │     │     │     │     │     │                       │
│   └─────┘     └─────┘     └─────┘     └─────┘                       │
│                                                                     │
│   4개를 쌓으면: "공이 오른쪽으로 이동 중" 을 알 수 있음!            │
│                                                                     │
│   최종 Shape: (4, 84, 84)                                           │
│               └─ 4 채널 (프레임 수)                                 │
│                    └─ 84×84 그레이스케일 이미지                     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 📊 데이터 크기

| 항목 | 값 | 설명 |
|:---|:---|:---|
| 단일 observation | (4, 84, 84) | 28,224 픽셀 |
| 데이터 타입 | uint8 | 0~255 정수 |
| 메모리 (1개) | ~27KB | 압축 없이 |
| 1 버퍼 (100K) | ~2.7GB | 100,000개 |
| 50 버퍼 전체 | ~135GB | 대용량! |

In [ ]:
# ============================================================
# 🎮 더미 궤적 데이터 생성
# ============================================================
# 실제 DQN replay가 없어도 학습할 수 있도록 더미 데이터를 생성합니다.
# 구조는 실제 데이터와 동일합니다.
# ============================================================

def create_dummy_trajectory(length=100):
    """
    Atari 스타일의 더미 궤적 생성
    
    Returns:
        dict with keys:
        - observations: (length, 4, 84, 84) - 4 stacked frames
        - actions: (length,) - discrete actions
        - rewards: (length,) - sparse rewards (0 or positive)
        - terminals: (length,) - episode end flags
    """
    return {
        # 84x84 그레이스케일 이미지 4개 스택
        'observations': np.random.randint(0, 256, (length, 4, 84, 84), dtype=np.uint8),
        
        # 이산 행동 (Breakout: 0~3)
        'actions': np.random.randint(0, 4, length),
        
        # Atari reward는 희소(sparse): 대부분 0, 가끔 양수
        'rewards': np.random.choice([0, 0, 0, 1, 5, 10], length).astype(np.float32),
        
        # 에피소드 종료 플래그
        'terminals': np.zeros(length, dtype=bool)
    }

# 여러 궤적 생성 (여러 에피소드)
print("🎮 더미 궤적 데이터 생성")
print("="*60)

trajectories = []
for i in range(10):
    # 각 에피소드는 50~200 스텝
    traj = create_dummy_trajectory(np.random.randint(50, 200))
    traj['terminals'][-1] = True  # 마지막 스텝은 terminal
    trajectories.append(traj)

print(f"생성된 에피소드 수: {len(trajectories)}")

# 첫 번째 궤적 분석
traj = trajectories[0]
print(f"\n📊 첫 번째 에피소드 분석:")
print(f"   ┌─ observations: {traj['observations'].shape}")
print(f"   │                └─ (에피소드 길이, 프레임 수, 높이, 너비)")
print(f"   ├─ actions:      {traj['actions'].shape}")
print(f"   │                └─ 이산 action (0~3)")
print(f"   ├─ rewards:      {traj['rewards'].shape}")
print(f"   │                └─ 희소 reward")
print(f"   └─ terminals:    {traj['terminals'].shape}")

# 메모리 사용량
memory_mb = traj['observations'].nbytes / (1024 * 1024)
print(f"\n💾 메모리 사용량: {memory_mb:.1f} MB (한 에피소드)")

# Reward 분포
unique, counts = np.unique(traj['rewards'], return_counts=True)
print(f"\n📈 Reward 분포:")
for u, c in zip(unique, counts):
    pct = c / len(traj['rewards']) * 100
    print(f"   reward={u:>2}: {c:>3}회 ({pct:>5.1f}%)")

## 2.2 RTG 계산 (Atari 스타일)

> Atari에서 RTG 계산 시 중요한 점: **에피소드 경계를 넘지 않는다!**

### ⚠️ 에피소드 경계 처리

```
┌─────────────────────────────────────────────────────────────────────┐
│                    에피소드 경계와 RTG                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  에피소드 1              에피소드 2                                  │
│  ───────────────────     ──────────────────────                     │
│  [r0, r1, r2, r3]        [r4, r5, r6]                               │
│  [ 1,  2,  3,  4]        [ 5, 10, 15]                               │
│                                                                     │
│  ❌ 잘못된 계산 (경계 무시):                                         │
│     RTG[3] = r3 + r4 + r5 + r6 = 4 + 5 + 10 + 15 = 34 (틀림!)      │
│                                                                     │
│  ✅ 올바른 계산 (경계 존중):                                         │
│     RTG[3] = r3 = 4 (에피소드 1의 마지막)                           │
│     RTG[4] = r4 + r5 + r6 = 30 (에피소드 2의 처음)                  │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 💡 왜 경계를 존중해야 할까?

- **에피소드 1에서 죽으면** → 에피소드 2의 보상을 받을 수 없음
- **RTG는 "앞으로 받을 수 있는 보상"** 을 의미
- 죽은 후에는 아무것도 받을 수 없으므로 **에피소드 내에서만** 계산

In [ ]:
# ============================================================
# 📈 Atari 스타일 RTG 계산 함수
# ============================================================
# create_dataset.py에서 사용하는 RTG 계산 방식입니다.
# 핵심: 에피소드 경계(done_idxs)를 존중하여 계산합니다.
# ============================================================

def calculate_rtg_atari_style(stepwise_returns, done_idxs):
    """
    Atari 스타일 RTG 계산
    
    Parameters:
    -----------
    stepwise_returns : np.ndarray
        모든 스텝의 reward (여러 에피소드가 연결된 형태)
    done_idxs : list
        각 에피소드가 끝나는 인덱스 (exclusive)
        예: [4, 7] → 에피소드1은 인덱스 0~3, 에피소드2는 4~6
        
    Returns:
    --------
    np.ndarray
        에피소드 경계를 존중하는 RTG
    """
    rtg = np.zeros_like(stepwise_returns)
    start_index = 0
    
    for done_idx in done_idxs:
        done_idx = int(done_idx)
        
        # 현재 에피소드의 reward만 추출
        curr_traj_returns = stepwise_returns[start_index:done_idx]
        
        # 역방향으로 RTG 계산 (끝에서 시작으로)
        for j in range(done_idx - 1, start_index - 1, -1):
            # j부터 에피소드 끝까지의 reward 합
            rtg_j = curr_traj_returns[j - start_index:done_idx - start_index]
            rtg[j] = sum(rtg_j)
        
        # 다음 에피소드로 이동
        start_index = done_idx
    
    return rtg

# ============================================================
# 🧪 RTG 계산 테스트
# ============================================================
print("📈 Atari 스타일 RTG 계산 예시")
print("="*60)

# 두 에피소드 생성
rewards_ep1 = [1, 2, 3, 4]    # 에피소드 1: 총 reward = 10
rewards_ep2 = [5, 10, 15]     # 에피소드 2: 총 reward = 30

# 연결
all_rewards = np.array(rewards_ep1 + rewards_ep2, dtype=np.float32)

# 에피소드 경계 인덱스 (exclusive)
done_idxs = [len(rewards_ep1), len(rewards_ep1) + len(rewards_ep2)]  # [4, 7]

# RTG 계산
rtg = calculate_rtg_atari_style(all_rewards, done_idxs)

print(f"\n입력 데이터:")
print(f"   rewards:    {all_rewards}")
print(f"   done_idxs:  {done_idxs}")

print(f"\n계산된 RTG:")
print(f"   rtg:        {rtg}")

print(f"\n📊 에피소드별 분석:")
print(f"   ┌─ 에피소드 1 (idx 0~3):")
print(f"   │    rewards: {rewards_ep1}")
print(f"   │    RTG:     {rtg[:4].tolist()}")
print(f"   │    └─ idx 0의 RTG = 1+2+3+4 = {rtg[0]}")
print(f"   │")
print(f"   └─ 에피소드 2 (idx 4~6):")
print(f"        rewards: {rewards_ep2}")
print(f"        RTG:     {rtg[4:].tolist()}")
print(f"        └─ idx 4의 RTG = 5+10+15 = {rtg[4]}")

In [ ]:
# ============================================================
# 📊 RTG 계산 시각화
# ============================================================
# 에피소드 경계에서 RTG가 어떻게 변화하는지 시각화합니다.
# ============================================================

print("📊 RTG 시각화")
print("="*60)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 색상 지정: 에피소드별로 다른 색
colors = ['steelblue'] * 4 + ['coral'] * 3

# 1. Reward 시각화
ax1 = axes[0]
bars1 = ax1.bar(range(len(all_rewards)), all_rewards, color=colors, alpha=0.7, edgecolor='black')
ax1.axvline(x=3.5, color='black', linestyle='--', linewidth=2, label='Episode boundary')
ax1.set_xlabel('Step Index', fontsize=12)
ax1.set_ylabel('Reward', fontsize=12)
ax1.set_title('Step-wise Reward (에피소드별 색상 구분)', fontsize=14)
ax1.set_xticks(range(len(all_rewards)))
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# 값 표시
for i, (r, bar) in enumerate(zip(all_rewards, bars1)):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
             f'{int(r)}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. RTG 시각화
ax2 = axes[1]
bars2 = ax2.bar(range(len(rtg)), rtg, color=colors, alpha=0.7, edgecolor='black')
ax2.axvline(x=3.5, color='black', linestyle='--', linewidth=2, label='Episode boundary')
ax2.set_xlabel('Step Index', fontsize=12)
ax2.set_ylabel('Return-to-Go', fontsize=12)
ax2.set_title('RTG (에피소드 내에서만 계산)', fontsize=14)
ax2.set_xticks(range(len(rtg)))
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# 값 표시
for i, (r, bar) in enumerate(zip(rtg, bars2)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{int(r)}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# 핵심 포인트
print("\n💡 핵심 포인트:")
print("   • 에피소드 1의 RTG는 에피소드 1 내에서만 계산됨")
print("   • 에피소드 2의 RTG는 에피소드 2 내에서만 계산됨")
print("   • 경계(인덱스 3→4)에서 RTG가 급격히 변화")
print("   • 이는 '죽으면 다음 에피소드 보상을 받을 수 없다'는 의미 반영")

## 2.3 StateActionReturnDataset

> PyTorch Dataset으로 배치 학습을 위한 데이터 로더를 구현합니다.

### 📦 Dataset 구조

```
┌─────────────────────────────────────────────────────────────────────┐
│              StateActionReturnDataset 동작 방식                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  전체 데이터: [ep1 data][ep2 data][ep3 data]...                     │
│              ──────────────────────────────────                     │
│                                                                     │
│  __getitem__(idx):                                                  │
│  ┌────────────────────────────────────────────────────────────────┐│
│  │ 1. idx 위치 확인                                               ││
│  │ 2. 에피소드 경계 체크 (done_idx 찾기)                          ││
│  │ 3. context_length만큼 추출 (경계 넘지 않게)                    ││
│  │ 4. 이미지 정규화 (0~255 → 0~1)                                ││
│  │ 5. (states, actions, rtgs, timesteps) 반환                     ││
│  └────────────────────────────────────────────────────────────────┘│
│                                                                     │
│  반환 형태:                                                         │
│    states:    (context_len, 4*84*84) - 평탄화된 이미지             │
│    actions:   (context_len, 1) - 이산 액션                         │
│    rtgs:      (context_len, 1) - RTG                               │
│    timesteps: (1, 1) - 시작 시간                                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
# 📦 StateActionReturnDataset 구현
# ============================================================
# run_dt_atari.py에서 사용하는 Dataset 클래스입니다.
# 에피소드 경계를 존중하면서 context_length 크기의 시퀀스를 반환합니다.
# ============================================================

from torch.utils.data import Dataset, DataLoader

class StateActionReturnDataset(Dataset):
    """
    Atari용 Dataset (run_dt_atari.py 참조)
    
    Parameters:
    -----------
    data : list
        observation 리스트 (각 요소는 (4, 84, 84) 이미지)
    block_size : int
        context_length * 3 (RTG, State, Action 세 종류)
    actions : np.ndarray
        action 배열
    done_idxs : list
        에피소드 종료 인덱스
    rtgs : np.ndarray
        RTG 배열
    timesteps : np.ndarray
        각 스텝의 에피소드 내 시간
    """
    
    def __init__(self, data, block_size, actions, done_idxs, rtgs, timesteps):
        self.block_size = block_size  # context_length * 3
        self.vocab_size = int(max(actions)) + 1  # action 수
        self.data = data
        self.actions = actions
        self.done_idxs = done_idxs
        self.rtgs = rtgs
        self.timesteps = timesteps
        
    def __len__(self):
        # context_length를 확보할 수 있는 샘플 수
        return len(self.data) - self.block_size // 3
    
    def __getitem__(self, idx):
        """
        idx 위치에서 context_length 크기의 시퀀스 반환
        """
        block_size = self.block_size // 3  # 실제 타임스텝 수
        done_idx = idx + block_size
        
        # 에피소드 경계 확인: 현재 idx보다 큰 첫 번째 done_idx 찾기
        for i in self.done_idxs:
            if i > idx:
                # 에피소드 끝을 넘지 않도록 조정
                done_idx = min(int(i), done_idx)
                break
        
        # 시작 인덱스 조정 (block_size 확보)
        idx = done_idx - block_size
        
        # ===== States =====
        # 이미지를 평탄화하고 정규화 (0~255 → 0~1)
        states = torch.tensor(
            np.array(self.data[idx:done_idx]), 
            dtype=torch.float32
        ).reshape(block_size, -1) / 255.0  # 정규화!
        
        # ===== Actions =====
        actions = torch.tensor(
            self.actions[idx:done_idx], 
            dtype=torch.long
        ).unsqueeze(1)
        
        # ===== RTGs =====
        rtgs = torch.tensor(
            self.rtgs[idx:done_idx], 
            dtype=torch.float32
        ).unsqueeze(1)
        
        # ===== Timesteps =====
        # 시작 timestep만 반환 (나머지는 모델에서 계산)
        timesteps = torch.tensor(
            self.timesteps[idx:idx+1], 
            dtype=torch.int64
        ).unsqueeze(1)
        
        return states, actions, rtgs, timesteps

# ============================================================
# 🧪 Dataset 테스트
# ============================================================
print("🧪 StateActionReturnDataset 테스트")
print("="*60)

# 더미 데이터 준비
num_samples = 500
context_length = 30

# 이미지 데이터 (4 stacked frames)
dummy_data = [np.random.randint(0, 256, (4, 84, 84), dtype=np.uint8) for _ in range(num_samples)]
dummy_actions = np.random.randint(0, 4, num_samples)
dummy_rewards = np.random.choice([0, 1, 5], num_samples).astype(np.float32)

# 에피소드 경계 (4개 에피소드)
dummy_done_idxs = [100, 200, 350, 500]

# RTG 계산
dummy_rtgs = calculate_rtg_atari_style(dummy_rewards, dummy_done_idxs)

# Timesteps (에피소드 내 시간)
dummy_timesteps = np.zeros(num_samples, dtype=np.int64)
start = 0
for done in dummy_done_idxs:
    dummy_timesteps[start:done] = np.arange(done - start)
    start = done

# Dataset 생성
dataset = StateActionReturnDataset(
    dummy_data, 
    context_length * 3,  # block_size
    dummy_actions, 
    dummy_done_idxs, 
    dummy_rtgs, 
    dummy_timesteps
)

print(f"\n📊 Dataset 정보:")
print(f"   총 샘플 수: {len(dataset)}")
print(f"   Vocab size (action 수): {dataset.vocab_size}")
print(f"   Block size: {dataset.block_size}")
print(f"   Context length: {context_length}")

# 샘플 하나 가져오기
states, actions, rtgs, timesteps = dataset[0]

print(f"\n📦 샘플 Shapes:")
print(f"   states:    {states.shape}")
print(f"              └─ ({context_length}, 4×84×84) = ({context_length}, 28224)")
print(f"   actions:   {actions.shape}")
print(f"              └─ ({context_length}, 1)")
print(f"   rtgs:      {rtgs.shape}")
print(f"              └─ ({context_length}, 1)")
print(f"   timesteps: {timesteps.shape}")
print(f"              └─ (1, 1)")

print(f"\n📈 데이터 범위:")
print(f"   states: [{states.min():.3f}, {states.max():.3f}] (정규화됨)")
print(f"   actions: {actions.unique().tolist()} (이산값)")
print(f"   rtgs: [{rtgs.min():.1f}, {rtgs.max():.1f}]")

---
# 3. CNN Encoder 이해

## 🎯 목표
> Atari 이미지를 처리하는 CNN Encoder의 구조와 동작을 이해합니다.

## 🖼️ 왜 CNN이 필요한가?

**Gym 환경 (Phase 3):**
- State: 11차원 벡터
- 직접 Linear 레이어로 임베딩 가능

**Atari 환경:**
- State: 84×84×4 = **28,224**차원 이미지
- 이미지의 공간적 구조를 활용해야 함
- **CNN으로 특징 추출** 후 임베딩

## 🏗️ CNN Encoder 구조

```
┌────────────────────────────────────────────────────────────────────────────┐
│                        CNN Encoder 아키텍처                                │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  입력: (batch, 4, 84, 84) - 4 stacked grayscale frames                    │
│                                                                            │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │ Conv1: Conv2d(4→32, kernel=8, stride=4)                             │  │
│  │        (batch, 4, 84, 84) → (batch, 32, 20, 20)                     │  │
│  │        + ReLU                                                       │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                              ↓                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │ Conv2: Conv2d(32→64, kernel=4, stride=2)                            │  │
│  │        (batch, 32, 20, 20) → (batch, 64, 9, 9)                      │  │
│  │        + ReLU                                                       │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                              ↓                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │ Conv3: Conv2d(64→64, kernel=3, stride=1)                            │  │
│  │        (batch, 64, 9, 9) → (batch, 64, 7, 7)                        │  │
│  │        + ReLU                                                       │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                              ↓                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │ Flatten: (batch, 64, 7, 7) → (batch, 3136)                          │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                              ↓                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │ Linear: Linear(3136 → n_embd) + Tanh                                │  │
│  │         (batch, 3136) → (batch, 128)                                │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                                                                            │
│  출력: (batch, 128) - State 임베딩 벡터                                    │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
```

> 💡 **이 구조는 DQN 논문의 CNN과 동일합니다!** (Atari 강화학습의 표준)

In [ ]:
# ============================================================
# 🏗️ CNN Encoder 구현
# ============================================================
# model_atari.py의 CNN Encoder를 구현합니다.
# DQN 논문과 동일한 구조입니다.
# ============================================================

class AtariCNNEncoder(nn.Module):
    """
    DQN 스타일 CNN Encoder
    
    Parameters:
    -----------
    n_embd : int
        출력 임베딩 차원 (default: 128)
        
    입력: (batch, 4, 84, 84) - 4 stacked grayscale frames
    출력: (batch, n_embd) - State 임베딩 벡터
    """
    
    def __init__(self, n_embd=128):
        super().__init__()
        
        self.encoder = nn.Sequential(
            # ===== Conv1 =====
            # 입력: (4, 84, 84) → 출력: (32, 20, 20)
            # 계산: (84 - 8) / 4 + 1 = 20
            nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            
            # ===== Conv2 =====
            # 입력: (32, 20, 20) → 출력: (64, 9, 9)
            # 계산: (20 - 4) / 2 + 1 = 9
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            
            # ===== Conv3 =====
            # 입력: (64, 9, 9) → 출력: (64, 7, 7)
            # 계산: (9 - 3) / 1 + 1 = 7
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            
            # ===== Flatten =====
            # 입력: (64, 7, 7) → 출력: (3136)
            # 계산: 64 × 7 × 7 = 3136
            nn.Flatten(),
            
            # ===== Linear =====
            # 입력: (3136) → 출력: (n_embd)
            nn.Linear(64 * 7 * 7, n_embd),
            nn.Tanh()  # 출력을 [-1, 1] 범위로 정규화
        )
        
    def forward(self, x):
        """
        Args:
            x: (batch, 4, 84, 84) - 4 stacked grayscale frames
            
        Returns:
            (batch, n_embd) - State 임베딩 벡터
        """
        return self.encoder(x)

# ============================================================
# 🧪 CNN Encoder 테스트
# ============================================================
print("🏗️ CNN Encoder 테스트")
print("="*60)

encoder = AtariCNNEncoder(n_embd=128)

print("\n📦 모델 구조:")
print(encoder)

# 파라미터 수 계산
total_params = sum(p.numel() for p in encoder.parameters())
print(f"\n📊 총 파라미터 수: {total_params:,}")

# Forward pass 테스트
x = torch.randn(4, 4, 84, 84)  # batch=4
out = encoder(x)

print(f"\n🔄 Forward Pass:")
print(f"   입력: {x.shape}")
print(f"   출력: {out.shape}")
print(f"\n   차원 축소 비율: {4*84*84} → {out.shape[-1]}")
print(f"   ({4*84*84 / out.shape[-1]:.0f}배 압축)")

In [ ]:
# ============================================================
# 📐 각 레이어의 출력 크기 추적
# ============================================================
# CNN에서 출력 크기 계산 공식:
#   output_size = (input_size - kernel_size + 2*padding) / stride + 1
#
# 이 공식을 사용해 각 레이어의 출력을 추적합니다.
# ============================================================

def calc_conv_output(input_size, kernel_size, stride, padding=0):
    """Conv2d 출력 크기 계산"""
    return (input_size - kernel_size + 2 * padding) // stride + 1

print("📐 CNN 출력 크기 단계별 추적")
print("="*60)

# 단계별 출력 기록
stages = []

# 입력
size = 84
channels = 4
stages.append(('Input', channels, size, size))

# Conv1
channels = 32
size = calc_conv_output(size, kernel_size=8, stride=4)
stages.append(('Conv1 (k=8, s=4)', channels, size, size))

# Conv2
channels = 64
size = calc_conv_output(size, kernel_size=4, stride=2)
stages.append(('Conv2 (k=4, s=2)', channels, size, size))

# Conv3
channels = 64
size = calc_conv_output(size, kernel_size=3, stride=1)
stages.append(('Conv3 (k=3, s=1)', channels, size, size))

# 표 형식으로 출력
print(f"\n{'Stage':<20} {'Shape':<20} {'Elements':>12}")
print("-"*55)

for stage_name, c, h, w in stages:
    shape = f"({c}, {h}, {w})"
    elements = c * h * w
    print(f"{stage_name:<20} {shape:<20} {elements:>12,}")

# Flatten
flatten_size = stages[-1][1] * stages[-1][2] * stages[-1][3]
print(f"{'Flatten':<20} {'(' + str(flatten_size) + ')':<20} {flatten_size:>12,}")

# Linear
final_size = 128
print(f"{'Linear':<20} {'(' + str(final_size) + ')':<20} {final_size:>12,}")

print("\n" + "-"*55)
print(f"총 압축률: {4*84*84:,} → {final_size} ({4*84*84/final_size:.0f}배)")

In [ ]:
# ============================================================
# 🔍 실제 텐서로 추적 확인
# ============================================================
# 실제로 텐서를 통과시키며 각 레이어의 출력을 확인합니다.
# ============================================================

def trace_cnn_with_tensor(x):
    """
    실제 텐서로 CNN 각 단계를 추적
    """
    print(f"🔍 실제 텐서로 CNN 추적")
    print("="*60)
    print(f"입력: {x.shape}")
    print("-"*60)
    
    # Conv1
    conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
    x = F.relu(conv1(x))
    print(f"Conv1 (k=8, s=4) + ReLU: {x.shape}")
    print(f"  └─ 계산: (84-8)/4 + 1 = 20")
    
    # Conv2
    conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
    x = F.relu(conv2(x))
    print(f"Conv2 (k=4, s=2) + ReLU: {x.shape}")
    print(f"  └─ 계산: (20-4)/2 + 1 = 9")
    
    # Conv3
    conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
    x = F.relu(conv3(x))
    print(f"Conv3 (k=3, s=1) + ReLU: {x.shape}")
    print(f"  └─ 계산: (9-3)/1 + 1 = 7")
    
    # Flatten
    x = x.flatten(1)  # batch 차원 유지
    print(f"Flatten: {x.shape}")
    print(f"  └─ 64 × 7 × 7 = 3136")
    
    # Linear
    linear = nn.Linear(3136, 128)
    x = torch.tanh(linear(x))
    print(f"Linear + Tanh: {x.shape}")
    
    print("-"*60)
    print(f"최종 출력: {x.shape}")
    return x

# 테스트 실행
test_input = torch.randn(1, 4, 84, 84)  # batch=1
output = trace_cnn_with_tensor(test_input)

---
# 4. 모델 구조 분석

## 🎯 목표
> Atari용 Decision Transformer (AtariGPT)의 전체 구조를 이해합니다.

## 🏗️ AtariGPT 전체 아키텍처

```
┌────────────────────────────────────────────────────────────────────────────┐
│                         AtariGPT 전체 구조                                  │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  입력: states (B, K, 4*84*84), actions (B, K, 1), rtgs (B, K, 1)          │
│                                                                            │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │                        Embedding Layer                              │  │
│  │  ┌────────────┐   ┌────────────┐   ┌────────────┐                  │  │
│  │  │ CNN        │   │ RTG        │   │ Action     │                  │  │
│  │  │ Encoder    │   │ Embedding  │   │ Embedding  │                  │  │
│  │  │ (state)    │   │ (Linear)   │   │ (Lookup)   │                  │  │
│  │  └─────┬──────┘   └─────┬──────┘   └─────┬──────┘                  │  │
│  │        │                │                │                          │  │
│  │        ▼                ▼                ▼                          │  │
│  │     (B,K,128)        (B,K,128)        (B,K,128)                     │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                                │                                          │
│                                ▼                                          │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │  Sequence Construction (Reward-Conditioned)                         │  │
│  │  [R₀, s₀, a₀, R₁, s₁, a₁, ..., Rₖ, sₖ, aₖ]                         │  │
│  │                                                                     │  │
│  │  Shape: (B, K*3, 128) = (B, 90, 128) when K=30                     │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                                │                                          │
│                                ▼                                          │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │  Transformer Blocks (× N layers)                                    │  │
│  │  ┌─────────────────────────────────────────────────────────────┐   │  │
│  │  │ LayerNorm → CausalSelfAttention → + → LayerNorm → MLP → +   │   │  │
│  │  └─────────────────────────────────────────────────────────────┘   │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                                │                                          │
│                                ▼                                          │
│  ┌─────────────────────────────────────────────────────────────────────┐  │
│  │  Prediction Head                                                    │  │
│  │  Linear(128 → vocab_size) at state positions (1::3)                │  │
│  │  → action logits (분류 문제!)                                       │  │
│  └─────────────────────────────────────────────────────────────────────┘  │
│                                                                            │
│  손실: Cross-Entropy (action 분류)                                         │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
```

## 💡 Gym vs Atari 손실 함수 비교

| 환경 | Action 타입 | 손실 함수 | 출력 |
|:---|:---|:---|:---|
| **Gym** | 연속값 | MSE | (B, K, action_dim) |
| **Atari** | 이산값 | Cross-Entropy | (B, K, vocab_size) logits |

In [ ]:
# ============================================================
# 🏗️ Transformer 구성 요소 구현
# ============================================================
# model_atari.py의 핵심 컴포넌트들을 구현합니다.
# - CausalSelfAttention: 인과적 self-attention
# - Block: Transformer 블록
# ============================================================

import math

class CausalSelfAttention(nn.Module):
    """
    Causal Self-Attention (미래 정보를 보지 않는 attention)
    
    GPT 스타일의 unidirectional attention입니다.
    현재 토큰은 이전 토큰들만 참조할 수 있습니다.
    """
    
    def __init__(self, n_embd, n_head, block_size, attn_pdrop=0.1, resid_pdrop=0.1):
        super().__init__()
        assert n_embd % n_head == 0, "n_embd must be divisible by n_head"
        
        # Q, K, V projection
        self.key = nn.Linear(n_embd, n_embd)
        self.query = nn.Linear(n_embd, n_embd)
        self.value = nn.Linear(n_embd, n_embd)
        
        # Dropout layers
        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)
        
        # Output projection
        self.proj = nn.Linear(n_embd, n_embd)
        
        # Causal mask: 하삼각 행렬 (미래 정보 차단)
        # mask[i,j] = 1 if j <= i else 0
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(block_size + 1, block_size + 1))
                 .view(1, 1, block_size + 1, block_size + 1)
        )
        
        self.n_head = n_head
        
    def forward(self, x):
        """
        Args:
            x: (B, T, C) - 입력 시퀀스
            
        Returns:
            (B, T, C) - attention 적용 후 출력
        """
        B, T, C = x.size()
        
        # Q, K, V 계산 후 multi-head로 reshape
        # (B, T, C) → (B, T, n_head, head_dim) → (B, n_head, T, head_dim)
        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        
        # Attention 계산
        # (B, n_head, T, head_dim) @ (B, n_head, head_dim, T) → (B, n_head, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        
        # Causal masking: 미래 위치에 -inf 적용
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
        
        # Softmax
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        
        # Value와 곱하기
        # (B, n_head, T, T) @ (B, n_head, T, head_dim) → (B, n_head, T, head_dim)
        y = att @ v
        
        # Reshape back
        # (B, n_head, T, head_dim) → (B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # Output projection
        y = self.resid_drop(self.proj(y))
        
        return y


class Block(nn.Module):
    """
    Transformer Block
    
    구조: LayerNorm → Attention → + → LayerNorm → MLP → +
    (Pre-LN 구조)
    """
    
    def __init__(self, n_embd, n_head, block_size, resid_pdrop=0.1):
        super().__init__()
        
        # LayerNorm layers
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
        # Self-Attention
        self.attn = CausalSelfAttention(n_embd, n_head, block_size)
        
        # MLP (Feed-Forward Network)
        # hidden dim = 4 * n_embd (GPT-2 표준)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(resid_pdrop),
        )
        
    def forward(self, x):
        """
        Args:
            x: (B, T, C)
            
        Returns:
            (B, T, C)
        """
        # Attention with residual connection
        x = x + self.attn(self.ln1(x))
        
        # MLP with residual connection
        x = x + self.mlp(self.ln2(x))
        
        return x

# ============================================================
# 🧪 Block 테스트
# ============================================================
print("🏗️ Transformer Block 테스트")
print("="*60)

block = Block(n_embd=128, n_head=8, block_size=90)
x = torch.randn(4, 90, 128)  # (batch, seq_len, hidden)

out = block(x)

print(f"입력: {x.shape}")
print(f"출력: {out.shape}")
print(f"잔차 연결로 shape 유지됨 ✓")

In [ ]:
# ============================================================
# 🎮 AtariGPT 전체 모델 구현
# ============================================================
# model_atari.py의 GPT 클래스를 간소화하여 구현합니다.
# 두 가지 모드를 지원합니다:
#   - reward_conditioned: Decision Transformer (RTG 사용)
#   - naive: Behavior Cloning (RTG 미사용)
# ============================================================

class AtariGPT(nn.Module):
    """
    Atari용 Decision Transformer
    
    Parameters:
    -----------
    vocab_size : int
        Action 수 (게임마다 다름: Breakout=4, Seaquest=18)
    block_size : int
        총 시퀀스 길이 (context_length * 3)
    n_layer : int
        Transformer 레이어 수
    n_head : int
        Attention head 수
    n_embd : int
        Hidden dimension
    max_timestep : int
        에피소드 내 최대 timestep
    model_type : str
        'reward_conditioned' (DT) or 'naive' (BC)
    """
    
    def __init__(self, vocab_size, block_size, n_layer=6, n_head=8, n_embd=128,
                 max_timestep=4000, model_type='reward_conditioned'):
        super().__init__()
        
        self.model_type = model_type
        self.block_size = block_size
        self.n_embd = n_embd
        
        # ===== Position Embeddings =====
        # 시퀀스 내 위치 임베딩
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size + 1, n_embd))
        # 에피소드 내 절대 시간 임베딩
        self.global_pos_emb = nn.Parameter(torch.zeros(1, max_timestep + 1, n_embd))
        
        self.drop = nn.Dropout(0.1)
        
        # ===== Transformer Blocks =====
        self.blocks = nn.Sequential(*[
            Block(n_embd, n_head, block_size) for _ in range(n_layer)
        ])
        self.ln_f = nn.LayerNorm(n_embd)
        
        # ===== Prediction Head =====
        # Action 분류 (logits 출력)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)
        
        # ===== Encoders =====
        # State: CNN Encoder
        self.state_encoder = AtariCNNEncoder(n_embd)
        
        # RTG: Linear + Tanh
        self.ret_emb = nn.Sequential(nn.Linear(1, n_embd), nn.Tanh())
        
        # Action: Embedding Lookup + Tanh
        self.action_embeddings = nn.Sequential(
            nn.Embedding(vocab_size, n_embd),
            nn.Tanh()
        )
        
    def forward(self, states, actions, targets=None, rtgs=None, timesteps=None):
        """
        Forward pass
        
        Args:
            states: (B, K, 4*84*84) - 평탄화된 이미지
            actions: (B, K, 1) - 이산 action
            targets: (B, K) - 정답 action (손실 계산용)
            rtgs: (B, K, 1) - RTG
            timesteps: (B, 1, 1) - 시작 timestep
            
        Returns:
            logits: (B, K, vocab_size) - action logits
            loss: scalar or None
        """
        batch_size = states.shape[0]
        seq_len = states.shape[1]  # K = context_length
        
        # ===== State Embedding via CNN =====
        # (B, K, 4*84*84) → (B*K, 4, 84, 84)
        states_reshaped = states.reshape(-1, 4, 84, 84)
        # CNN 통과
        state_embeddings = self.state_encoder(states_reshaped)
        # (B*K, n_embd) → (B, K, n_embd)
        state_embeddings = state_embeddings.reshape(batch_size, seq_len, self.n_embd)
        
        # ===== 모드별 시퀀스 구성 =====
        if self.model_type == 'reward_conditioned':
            # Decision Transformer: [R, s, a, R, s, a, ...]
            rtg_embeddings = self.ret_emb(rtgs.float())
            action_embeddings = self.action_embeddings(actions.squeeze(-1).long())
            
            # 시퀀스 구성: 3K 길이
            token_embeddings = torch.zeros(
                batch_size, seq_len * 3, self.n_embd,
                device=states.device
            )
            token_embeddings[:, 0::3, :] = rtg_embeddings     # 위치 0, 3, 6, ...
            token_embeddings[:, 1::3, :] = state_embeddings   # 위치 1, 4, 7, ...
            token_embeddings[:, 2::3, :] = action_embeddings  # 위치 2, 5, 8, ...
            
        else:  # naive (Behavior Cloning)
            # BC: [s, a, s, a, ...]
            action_embeddings = self.action_embeddings(actions.squeeze(-1).long())
            
            # 시퀀스 구성: 2K 길이
            token_embeddings = torch.zeros(
                batch_size, seq_len * 2, self.n_embd,
                device=states.device
            )
            token_embeddings[:, 0::2, :] = state_embeddings   # 위치 0, 2, 4, ...
            token_embeddings[:, 1::2, :] = action_embeddings  # 위치 1, 3, 5, ...
        
        # ===== Position Embedding 추가 =====
        position_embeddings = self.pos_emb[:, :token_embeddings.shape[1], :]
        
        # ===== Transformer 통과 =====
        x = self.drop(token_embeddings + position_embeddings)
        x = self.blocks(x)
        x = self.ln_f(x)
        
        # ===== Prediction Head =====
        logits = self.head(x)
        
        # ===== State 위치의 출력만 사용 =====
        if self.model_type == 'reward_conditioned':
            logits = logits[:, 1::3, :]  # State 위치 (1, 4, 7, ...)
        else:
            logits = logits[:, 0::2, :]  # State 위치 (0, 2, 4, ...)
        
        # ===== 손실 계산 =====
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.reshape(-1, logits.size(-1)),
                targets.reshape(-1)
            )
        
        return logits, loss

# ============================================================
# 🧪 AtariGPT 테스트
# ============================================================
print("🎮 AtariGPT 모델 생성 및 테스트")
print("="*60)

model = AtariGPT(
    vocab_size=4,       # Breakout: 4 actions
    block_size=90,      # context_length * 3 = 30 * 3
    n_layer=6,
    n_head=8,
    n_embd=128,
    model_type='reward_conditioned'
)

# 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 모델 통계:")
print(f"   총 파라미터: {total_params:,}")
print(f"   모델 타입: {model.model_type}")

In [ ]:
# ============================================================
# 🔄 Forward Pass 테스트
# ============================================================
# 모델에 더미 입력을 넣어 동작을 확인합니다.
# ============================================================

print("🔄 Forward Pass 테스트")
print("="*60)

batch_size = 2
context_len = 30

# 입력 생성 (정규화된 이미지)
states = torch.randn(batch_size, context_len, 4 * 84 * 84)     # 평탄화된 이미지
actions = torch.randint(0, 4, (batch_size, context_len, 1))    # 이산 action
rtgs = torch.randn(batch_size, context_len, 1)                  # RTG
timesteps = torch.zeros(batch_size, 1, 1, dtype=torch.long)    # 시작 timestep

print("\n📥 입력 Shapes:")
print(f"   states:    {states.shape}")
print(f"              └─ (batch, context_len, 4×84×84)")
print(f"   actions:   {actions.shape}")
print(f"              └─ (batch, context_len, 1)")
print(f"   rtgs:      {rtgs.shape}")
print(f"              └─ (batch, context_len, 1)")
print(f"   timesteps: {timesteps.shape}")
print(f"              └─ (batch, 1, 1)")

# Forward (손실 계산 포함)
model.eval()
with torch.no_grad():
    targets = actions.squeeze(-1)  # 정답 action
    logits, loss = model(states, actions, targets, rtgs, timesteps)

print(f"\n📤 출력 Shapes:")
print(f"   logits: {logits.shape}")
print(f"           └─ (batch, context_len, vocab_size={4})")
print(f"   loss:   {loss.item():.4f}")

# Logits → Action 변환 (argmax)
predicted_actions = logits.argmax(dim=-1)
print(f"\n🎯 예측된 Action (마지막 timestep):")
print(f"   batch 0: {predicted_actions[0, -1].item()}")
print(f"   batch 1: {predicted_actions[1, -1].item()}")

# Softmax로 확률 변환
probs = F.softmax(logits[0, -1], dim=-1)
print(f"\n📊 Action 확률 분포 (batch 0, 마지막 timestep):")
for i, p in enumerate(probs):
    bar = '█' * int(p * 20)
    print(f"   Action {i}: {p:.3f} {bar}")

---
# 5. Reward-Conditioned vs Naive 비교

## 🎯 목표
> Decision Transformer의 두 모드 (reward_conditioned, naive)의 차이를 이해합니다.

## 📊 두 모드 비교

```
┌────────────────────┬──────────────────────────────┬──────────────────────────────┐
│        항목        │   Reward-Conditioned (DT)    │         Naive (BC)           │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 시퀀스 구성        │ [R₀, s₀, a₀, R₁, s₁, a₁,...] │ [s₀, a₀, s₁, a₁, ...]        │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 시퀀스 길이        │ K × 3                        │ K × 2                        │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 예측 추출 위치     │ 1::3 (State 위치)            │ 0::2 (State 위치)            │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 조건화             │ ✅ RTG로 목표 return 지정   │ ❌ 조건 없음                 │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 게임별 목표 RTG    │ Breakout: 90                 │                              │
│                    │ Seaquest: 1150               │ 사용 안 함                   │
│                    │ Qbert: 14000                 │ (0으로 설정)                 │
│                    │ Pong: 20                     │                              │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 추론 시 RTG        │ 동적 업데이트 (rtg -= r)     │ 고정                         │
├────────────────────┼──────────────────────────────┼──────────────────────────────┤
│ 성능 조절          │ RTG를 높게 → 더 나은 성능    │ 조절 불가                    │
│                    │ RTG를 낮게 → 더 안전한 행동  │ (데이터 평균 성능)           │
└────────────────────┴──────────────────────────────┴──────────────────────────────┘
```

## 💡 핵심 차이

**Reward-Conditioned (DT):**
- "1000점을 얻고 싶다" → 1000점 얻을 때의 행동 예측
- 다양한 수준의 정책 생성 가능

**Naive (BC):**
- 단순히 데이터의 평균적인 행동 모방
- 조건 없이 무조건 모방

In [ ]:
# ============================================================
# 📊 두 모드 비교표 출력
# ============================================================
# ASCII 아트로 두 모드의 차이를 시각적으로 보여줍니다.
# ============================================================

comparison = """
┌────────────────────────────────────────────────────────────────────────────┐
│                    Reward-Conditioned vs Naive 비교                         │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  🎯 Reward-Conditioned (Decision Transformer)                              │
│  ─────────────────────────────────────────────                             │
│  시퀀스: [R₀, s₀, a₀, R₁, s₁, a₁, R₂, s₂, a₂, ...]                        │
│           ↑    ↑    ↑                                                      │
│          RTG  State Action                                                 │
│                                                                            │
│  예측:   R₀ → s₀ → a₀?    (RTG와 State가 주어지면 Action 예측)             │
│               ↑                                                            │
│          1::3 위치에서 추출                                                │
│                                                                            │
│  특징:                                                                     │
│    • 목표 RTG를 지정하면 그에 맞는 행동 예측                               │
│    • 높은 RTG → 고성능 정책, 낮은 RTG → 보수적 정책                        │
│    • "나는 90점을 원해!" → 90점 달성 시의 행동 예측                        │
│                                                                            │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  📋 Naive (Behavior Cloning)                                               │
│  ─────────────────────────────                                             │
│  시퀀스: [s₀, a₀, s₁, a₁, s₂, a₂, ...]                                     │
│           ↑    ↑                                                           │
│         State Action (RTG 없음!)                                           │
│                                                                            │
│  예측:   s₀ → a₀?    (State만 보고 Action 예측)                            │
│          ↑                                                                 │
│     0::2 위치에서 추출                                                     │
│                                                                            │
│  특징:                                                                     │
│    • RTG 조건 없이 단순 모방                                               │
│    • 데이터 평균 수준의 정책                                               │
│    • "그냥 데이터에서 본 대로 행동할게!"                                   │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
"""
print(comparison)

# 게임별 목표 RTG
print("\n📈 게임별 목표 RTG (run_dt_atari.py에서 사용):")
game_rtgs = {
    'Breakout': 90,
    'Pong': 20,
    'Qbert': 14000,
    'Seaquest': 1150,
}
print(f"{'게임':<15} {'목표 RTG':>10} {'설명':<30}")
print("-"*60)
for game, rtg in game_rtgs.items():
    if game == 'Breakout':
        desc = "벽돌 90개 깨기"
    elif game == 'Pong':
        desc = "21점 중 20점 획득"
    elif game == 'Qbert':
        desc = "높은 점수 목표"
    else:
        desc = "잠수함 게임"
    print(f"{game:<15} {rtg:>10} {desc:<30}")

In [ ]:
# ============================================================
# 📊 시퀀스 구성 시각화
# ============================================================
# 두 모드의 시퀀스 구성을 시각적으로 비교합니다.
# ============================================================

print("📊 시퀀스 구성 시각화")
print("="*60)

fig, axes = plt.subplots(2, 1, figsize=(14, 7))

# ===== Reward-Conditioned =====
rc_seq = ['R₀', 's₀', 'a₀', 'R₁', 's₁', 'a₁', 'R₂', 's₂', 'a₂']
rc_colors = ['#ff6b6b', '#4ecdc4', '#45b7d1'] * 3  # 빨강, 청록, 파랑

ax1 = axes[0]
for i, (token, color) in enumerate(zip(rc_seq, rc_colors)):
    rect = plt.Rectangle((i, 0), 0.9, 1, facecolor=color, edgecolor='black', alpha=0.8)
    ax1.add_patch(rect)
    ax1.text(i + 0.45, 0.5, token, ha='center', va='center', fontsize=12, fontweight='bold')
    
    # State 위치에서 예측 화살표 (1::3)
    if i % 3 == 1:
        ax1.annotate('', xy=(i + 0.45, -0.15), xytext=(i + 0.45, -0.4),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2))
        ax1.text(i + 0.45, -0.6, f'→a{i//3} 예측', ha='center', color='red', fontsize=10, fontweight='bold')

ax1.set_xlim(-0.5, len(rc_seq) + 0.5)
ax1.set_ylim(-1, 1.5)
ax1.set_title('🎯 Reward-Conditioned: [R, s, a] 트리플 × K\n예측 위치: 1::3 (State 위치)', fontsize=14)
ax1.axis('off')

# ===== Naive =====
naive_seq = ['s₀', 'a₀', 's₁', 'a₁', 's₂', 'a₂']
naive_colors = ['#4ecdc4', '#45b7d1'] * 3  # 청록, 파랑 (RTG 없음)

ax2 = axes[1]
for i, (token, color) in enumerate(zip(naive_seq, naive_colors)):
    rect = plt.Rectangle((i, 0), 0.9, 1, facecolor=color, edgecolor='black', alpha=0.8)
    ax2.add_patch(rect)
    ax2.text(i + 0.45, 0.5, token, ha='center', va='center', fontsize=12, fontweight='bold')
    
    # State 위치에서 예측 화살표 (0::2)
    if i % 2 == 0:
        ax2.annotate('', xy=(i + 0.45, -0.15), xytext=(i + 0.45, -0.4),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2))
        ax2.text(i + 0.45, -0.6, f'→a{i//2} 예측', ha='center', color='red', fontsize=10, fontweight='bold')

ax2.set_xlim(-0.5, len(naive_seq) + 0.5)
ax2.set_ylim(-1, 1.5)
ax2.set_title('📋 Naive (BC): [s, a] 페어 × K\n예측 위치: 0::2 (State 위치)', fontsize=14)
ax2.axis('off')

# 범례
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#ff6b6b', edgecolor='black', label='RTG (R)'),
    Patch(facecolor='#4ecdc4', edgecolor='black', label='State (s)'),
    Patch(facecolor='#45b7d1', edgecolor='black', label='Action (a)'),
]
fig.legend(handles=legend_elements, loc='upper right', ncol=3, fontsize=11)

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

print("\n💡 핵심 포인트:")
print("   • Reward-Conditioned는 RTG 토큰이 추가되어 시퀀스가 1.5배 길어짐")
print("   • 두 모드 모두 State 위치에서 다음 Action을 예측")
print("   • 예측 추출 인덱싱: RC는 1::3, Naive는 0::2")

In [ ]:
# ============================================================
# 📊 두 모델 파라미터 비교
# ============================================================
# Reward-Conditioned와 Naive 모델의 파라미터 수를 비교합니다.
# ============================================================

print("📊 모델 파라미터 비교")
print("="*60)

# 두 모델 생성
model_rc = AtariGPT(vocab_size=4, block_size=90, model_type='reward_conditioned')
model_naive = AtariGPT(vocab_size=4, block_size=60, model_type='naive')  # 2/3 길이

params_rc = sum(p.numel() for p in model_rc.parameters())
params_naive = sum(p.numel() for p in model_naive.parameters())

print(f"\n{'모델':<25} {'파라미터 수':>15}")
print("-"*45)
print(f"{'Reward-Conditioned':<25} {params_rc:>15,}")
print(f"{'Naive (BC)':<25} {params_naive:>15,}")
print(f"{'차이':<25} {params_rc - params_naive:>15,}")

print(f"\n📝 차이 발생 이유:")
print(f"   • Block size: RC={90} vs Naive={60}")
print(f"   • Position embedding 크기가 다름")
print(f"   • RTG embedding 레이어 (Naive에는 없음)")
print(f"   • 하지만 메인 Transformer 파라미터는 동일")

## 🎮 추론 시 RTG 업데이트

> Reward-Conditioned 모드에서는 추론 시 RTG를 동적으로 업데이트합니다.

### RTG 업데이트 과정

```
┌────────────────────────────────────────────────────────────────────────────┐
│                    Breakout 게임 추론 시뮬레이션                            │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  목표: 90점 (벽돌 90개)                                                    │
│                                                                            │
│  Step 0:  RTG = 90  →  Model  →  Action: 오른쪽                           │
│           │                       ↓                                        │
│           │              환경: reward = 0 (빗나감)                         │
│           └──────────────────────────────────────────────────────────────  │
│                                                                            │
│  Step 1:  RTG = 90 - 0 = 90  →  Model  →  Action: 공 발사                 │
│           │                       ↓                                        │
│           │              환경: reward = 5 (벽돌 5개 파괴!)                 │
│           └──────────────────────────────────────────────────────────────  │
│                                                                            │
│  Step 2:  RTG = 90 - 5 = 85  →  Model  →  Action: 왼쪽                    │
│           │                       ↓                                        │
│           │              환경: reward = 10 (벽돌 10개 파괴!)               │
│           └──────────────────────────────────────────────────────────────  │
│                                                                            │
│  Step 3:  RTG = 85 - 10 = 75  →  ...                                      │
│                                                                            │
│  ...계속...                                                                │
│                                                                            │
│  Step N:  RTG ≈ 0  (목표 달성!)                                           │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
```

### 💡 핵심 원리

- **RTG = 남은 목표**
- reward를 받으면 **RTG -= reward**
- 모델은 **"남은 RTG를 달성하려면 어떤 행동?"** 을 예측
- RTG가 0에 가까워지면 목표 달성!

In [ ]:
# ============================================================
# 🎮 Breakout 게임 추론 시뮬레이션
# ============================================================
# 실제 환경 없이 RTG 업데이트 과정을 시뮬레이션합니다.
# ============================================================

def simulate_breakout_inference(target_rtg=90, max_steps=30):
    """
    Breakout 게임에서의 RTG 기반 추론 시뮬레이션
    
    Parameters:
    -----------
    target_rtg : int
        목표 점수 (벽돌 개수)
    max_steps : int
        시뮬레이션 최대 스텝
        
    Returns:
    --------
    list: 각 스텝의 기록
    """
    rtg = target_rtg
    steps = []
    
    # Breakout 보상 패턴 시뮬레이션
    # 실제로는 벽돌을 깨면 1~7점 (색상별)
    reward_pattern = [0, 0, 0, 5, 0, 0, 0, 5, 0, 5,
                      0, 0, 10, 0, 0, 5, 0, 0, 0, 10,
                      0, 5, 0, 0, 5, 0, 0, 10, 0, 0]
    
    print(f"🎮 Breakout 추론 시뮬레이션")
    print(f"   목표 점수: {target_rtg}")
    print("="*60)
    
    total_reward = 0
    for t in range(min(max_steps, len(reward_pattern))):
        reward = reward_pattern[t]
        
        # 기록
        steps.append({
            't': t,
            'rtg_before': rtg,
            'reward': reward,
            'rtg_after': rtg - reward,
            'total_reward': total_reward + reward
        })
        
        # 벽돌을 깼을 때만 출력
        if reward > 0:
            action = np.random.choice(['←', '→', '○'])  # 가상 액션
            print(f"Step {t:2d}: RTG={rtg:>3} → Action={action} → 벽돌 파괴! reward={reward:>2} → 새 RTG={rtg-reward:>3}")
        
        # RTG 업데이트
        rtg = rtg - reward
        total_reward += reward
        
        # 목표 달성 체크
        if rtg <= 0:
            print()
            print(f"🎉 목표 달성! (RTG ≤ 0)")
            break
    
    print("-"*60)
    print(f"\n📊 결과 요약:")
    print(f"   총 스텝: {len(steps)}")
    print(f"   총 획득 점수: {total_reward}")
    print(f"   목표 대비: {total_reward/target_rtg*100:.1f}%")
    print(f"   최종 RTG: {rtg}")
    
    return steps

# 시뮬레이션 실행
steps = simulate_breakout_inference(target_rtg=90)

In [ ]:
# ============================================================
# 📈 RTG 변화 시각화
# ============================================================
# 추론 과정에서 RTG와 누적 보상이 어떻게 변화하는지 시각화합니다.
# ============================================================

print("📈 RTG 변화 시각화")
print("="*60)

# 데이터 추출
timesteps_viz = [s['t'] for s in steps]
rtgs_before = [s['rtg_before'] for s in steps]
rewards = [s['reward'] for s in steps]
total_rewards = [s['total_reward'] for s in steps]

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# ===== RTG 변화 =====
ax1 = axes[0]
ax1.plot(timesteps_viz, rtgs_before, 'o-', color='coral', linewidth=2, markersize=6, label='RTG')
ax1.axhline(y=0, color='green', linestyle='--', linewidth=2, label='Goal (RTG=0)')
ax1.fill_between(timesteps_viz, rtgs_before, 0, alpha=0.3, color='coral')

# 벽돌 파괴 지점 표시
hit_steps = [s['t'] for s in steps if s['reward'] > 0]
hit_rtgs = [s['rtg_before'] for s in steps if s['reward'] > 0]
ax1.scatter(hit_steps, hit_rtgs, color='red', s=100, zorder=5, label='Hit!')

ax1.set_ylabel('Return-to-Go', fontsize=12)
ax1.set_title('RTG Evolution During Inference (Breakout)', fontsize=14)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-1, len(steps))

# ===== 누적 보상 =====
ax2 = axes[1]

# 보상 막대 그래프
colors = ['green' if r > 0 else 'lightgray' for r in rewards]
ax2.bar(timesteps_viz, rewards, color=colors, alpha=0.7, label='Instant Reward')

# 누적 보상 선 그래프
ax2_twin = ax2.twinx()
ax2_twin.plot(timesteps_viz, total_rewards, 'b-', linewidth=2, marker='s', markersize=4, label='Cumulative Reward')
ax2_twin.set_ylabel('Cumulative Reward', fontsize=12, color='blue')
ax2_twin.tick_params(axis='y', labelcolor='blue')

ax2.set_xlabel('Step', fontsize=12)
ax2.set_ylabel('Instant Reward', fontsize=12)
ax2.set_title('Reward Accumulation (녹색 = 벽돌 파괴)', fontsize=14)
ax2.set_xlim(-1, len(steps))
ax2.legend(loc='upper left')
ax2_twin.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 핵심 인사이트
print("\n💡 핵심 인사이트:")
print(f"   • RTG는 reward를 받을 때마다 감소 (= 목표에 가까워짐)")
print(f"   • 누적 보상은 RTG의 역방향으로 증가")
print(f"   • RTG + 누적보상 ≈ 초기 목표 (= {steps[0]['rtg_before']})")

---
# 📝 연습 문제

> 이해도를 확인하기 위한 연습 문제입니다.

## 연습 1: CNN 출력 크기 계산

**문제**: 입력 이미지가 64×64 (84×84 대신)일 때, CNN 출력 크기는?

```
Conv1: kernel=8, stride=4
Conv2: kernel=4, stride=2
Conv3: kernel=3, stride=1
```

**공식**: `output = (input - kernel) / stride + 1`

# ============================================================
# 연습 1: 직접 계산해보세요!
# ============================================================
# TODO: 아래 코드를 완성하세요

def calc_output(input_size, kernel, stride):
    """Conv 출력 크기 계산"""
    return (input_size - kernel) // stride + 1

# 64x64 입력에 대한 계산
size = 64
print(f"입력: {size}x{size}")

# Conv1
# size = calc_output(size, ???, ???)
# print(f"Conv1: {size}x{size}")

# Conv2
# size = calc_output(size, ???, ???)
# print(f"Conv2: {size}x{size}")

# Conv3
# size = calc_output(size, ???, ???)
# print(f"Conv3: {size}x{size}")

# print(f"\nFlatten: 64 × {size} × {size} = {64 * size * size}")

In [ ]:
# ============================================================
# 📖 연습 1 정답
# ============================================================
# 아래 셀을 실행하여 정답을 확인하세요.
# ============================================================

print("📖 연습 1 정답")
print("="*40)

size = 64
print(f"입력: {size}x{size}")

# Conv1: (64-8)/4 + 1 = 15
size = calc_output(size, 8, 4)
print(f"Conv1 (k=8, s=4): {size}x{size}")

# Conv2: (15-4)/2 + 1 = 6
size = calc_output(size, 4, 2)
print(f"Conv2 (k=4, s=2): {size}x{size}")

# Conv3: (6-3)/1 + 1 = 4
size = calc_output(size, 3, 1)
print(f"Conv3 (k=3, s=1): {size}x{size}")

print(f"\nFlatten: 64 × {size} × {size} = {64 * size * size}")
print(f"\n💡 84x84일 때의 3136과 비교하면 {64 * size * size}은 더 작습니다!")

## 연습 2: 모드별 시퀀스 길이

**문제**: `context_length = 50`일 때, 각 모드의 실제 토큰 시퀀스 길이는?

| 모드 | 시퀀스 구성 | 길이 계산 | 정답 |
|:---|:---|:---|:---|
| Reward-Conditioned | [R, s, a, ...] | 50 × ? | ? |
| Naive | [s, a, ...] | 50 × ? | ? |

# ============================================================
# 연습 2: 직접 계산해보세요!
# ============================================================

context_length = 50

# TODO: 각 모드의 시퀀스 길이를 계산하세요
# reward_conditioned_seq_len = context_length * ???
# naive_seq_len = context_length * ???

# print(f"Reward-Conditioned: {reward_conditioned_seq_len}")
# print(f"Naive: {naive_seq_len}")

In [ ]:
# ============================================================
# 📖 연습 2 정답
# ============================================================

print("📖 연습 2 정답")
print("="*40)

context_length = 50

# Reward-conditioned: [R, s, a] 트리플 → 3배
reward_conditioned_seq_len = context_length * 3

# Naive: [s, a] 페어 → 2배
naive_seq_len = context_length * 2

print(f"Context length: {context_length}")
print()
print(f"Reward-Conditioned:")
print(f"   시퀀스 구성: [R, s, a, R, s, a, ...]")
print(f"   길이: {context_length} × 3 = {reward_conditioned_seq_len}")
print()
print(f"Naive (BC):")
print(f"   시퀀스 구성: [s, a, s, a, ...]")
print(f"   길이: {context_length} × 2 = {naive_seq_len}")
print()
print(f"💡 RC가 Naive보다 {reward_conditioned_seq_len - naive_seq_len}개 토큰 더 많음!")
print(f"   (RTG 토큰 {context_length}개가 추가됨)")

# 이 셀은 삭제됨 (연습문제 정답과 병합)
pass

---
# 🎓 Phase 4 완료!

## 🎉 축하합니다! 모든 Phase를 완료했습니다!

---

## 📚 이 Phase에서 배운 내용

### 1. DQN Replay Buffer
```
• 50개의 checkpoint (초보→전문가 정책)
• 각 100,000 transitions
• (observation, action, reward, terminal) 구조
• 4 stacked frames (84×84×4)
```

### 2. Atari 스타일 RTG 계산
```
• 에피소드 경계를 존중하여 계산
• done_idxs로 경계 관리
• 각 에피소드 내에서만 누적
```

### 3. CNN Encoder
```
입력: (4, 84, 84)
  ↓ Conv1 (k=8, s=4)
(32, 20, 20)
  ↓ Conv2 (k=4, s=2)
(64, 9, 9)
  ↓ Conv3 (k=3, s=1)
(64, 7, 7)
  ↓ Flatten
(3136)
  ↓ Linear
(128)
```

### 4. AtariGPT 구조
```
• CNN Encoder + Transformer
• Reward-Conditioned: [R, s, a] × K
• Naive (BC): [s, a] × K
• Cross-Entropy 손실 (이산 action)
```

### 5. 추론 시 RTG 업데이트
```
초기: RTG = 목표 점수
매 스텝: RTG = RTG - reward
종료: RTG ≈ 0 (목표 달성!)
```

---

## ✅ Phase 4 체크리스트

| 완료 | 항목 |
|:---:|:---|
| ☐ | DQN replay buffer 구조 이해 |
| ☐ | 4 frame stack의 의미 이해 |
| ☐ | Atari 스타일 RTG 계산 (에피소드 경계) 이해 |
| ☐ | StateActionReturnDataset 동작 이해 |
| ☐ | CNN Encoder 구조 및 출력 크기 계산 |
| ☐ | AtariGPT 전체 구조 이해 |
| ☐ | Reward-Conditioned vs Naive 차이 이해 |
| ☐ | 추론 시 RTG 동적 업데이트 이해 |

---

## 🚀 다음 단계 (선택)

### 1. 실제 데이터로 학습
```bash
cd atari
gsutil -m cp -R gs://atari-replay-datasets/dqn/Breakout dqn_replay/
python run_dt_atari.py --game Breakout --epochs 5
```

### 2. 하이퍼파라미터 실험
- context_length 변경 (30 → 50)
- n_layer 변경 (6 → 12)
- n_head 변경 (8 → 4)

### 3. 다른 게임 시도
- Pong, Qbert, Seaquest
- 게임별 목표 RTG 조정

### 4. 관련 논문 읽기
- Online Decision Transformer
- Trajectory Transformer
- Conservative Decision Transformer

---

## 🗺️ 전체 학습 로드맵 완료!

```
Phase 1 ──▶ Phase 2 ──▶ Phase 3 ──▶ Phase 4
  RL &        RTG &       Gym         Atari
Transformer  시퀀스      실습         실습
  기초        구성       (연속)      (이미지)
   ✅          ✅          ✅          ✅
```

---

> 🎉 **수고하셨습니다!**  
> Decision Transformer의 전체 구조와 동작 원리를 이해하셨습니다.  
> 이제 실제 데이터로 학습하고 실험해보세요!